In [1]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
datatype = "test"

In [3]:
if datatype == "train": 
    in_ad = pd.read_csv("/home/hassan/lily/MLA/FDA/in_ad.csv", index_col=0)
    death = pd.read_csv("/home/bhatti/dataset/VCHAMPS/death_train.csv", index_col=0)
if datatype == "test": 
    in_ad = pd.read_csv("/home/hassan/lily/MLA/FDA/in_ad_test.csv", index_col=0)
    death = pd.read_csv("/data/public/MLA/VCHAMPS-Test/death_test.csv", index_col=0)

In [4]:
in_ad["Died during admission"]

14        0
18        0
45        0
59        0
67        0
         ..
642994    0
658056    0
662481    0
663122    0
664654    0
Name: Died during admission, Length: 139086, dtype: int64

In [5]:
in_ad.columns

Index(['Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Admitting unit service', 'Discharging unit service',
       'Admitting specialty', 'Discharging specialty',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory',
       'Discharge disposition', 'Died during admission',
       'Outpatientreferralflag', 'Serviceconnectedflag', 'State', 'Transfer',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'stay_length', 'Age 20-40', 'Age 40-60', 'Age 60-80',
       'Age 80-100', 'Age 100-120', 'cd_diagnosis', 'Mental', 'Medical',
       'Rehab', 'Gerontology', 'Others_Specialty', 'Agentorangeflag'],
      dtype='object')

In [6]:
def final_set_before(ids, group):
    
    readmission = 0
    num_admissions = group["Age at admission"].nunique()
    if num_admissions > 1: readmission=1

    age_mean = group["Age at admission"].mean()
    age_std = group["Age at admission"].std()
    if group["Age at admission"].nunique() == 1: age_std = 0

    min_age = group["Age at admission"].min()
    max_age = group["Age at admission"].max()

    freq = num_admissions/(math.floor(max_age - min_age) + 1)

    min_stay = group["stay_length"].min()
    max_stay = group["stay_length"].min()
    stay_mean = group["stay_length"].mean()
    stay_std = group["stay_length"].std()
    if group["stay_length"].nunique() == 1: stay_std = 0

    # num_cvd_readmission = max(0, group['cd_diagnosis'].sum() - 1)
    num_cvd_admission = group['cd_diagnosis'].sum()

    cvd = 0
    if group["cd_diagnosis"].sum() > 0: cvd = 1
    
    Died = 0
    if group["Died during admission"].sum() > 0: Died = 1

    AO = 0
    if group["Agentorangeflag"].sum() > 0: AO = 1

    if (group.iloc[len(group)-1]["Died during admission"] == 1) and (group.iloc[len(group)-1]['cd_diagnosis'] == 1): died_by_cvd = 1
    else: died_by_cvd = 0

    df = pd.DataFrame(data={'Internalpatientid': [ids], 'num_stays': [num_admissions], 'stay_length': group["stay_length"].sum(),
                            'num_unique_units': group["Admitting unit service"].nunique(), "num_transfers": group["Transfer"].sum(), 
                            "num_cvd_admission": [num_cvd_admission], "Died": [Died], "AO": [AO], "CVD": [cvd],
                            "unique_admitting_specialty": group["Admitting specialty"].nunique(), "unique_discharging_specialty": group["Discharging specialty"].nunique(),
                            "DOMICILIARY": group["DOMICILIARY"].sum(), "MEDICINE": group["MEDICINE"].sum(), "NHCU":group["NHCU"].sum(),
                            "NON-COUNT":group["NON-COUNT"].sum(), "OTHERS":group["OTHERS"].sum(), 'PSYCHIATRY': group['PSYCHIATRY'].sum(), 'SURGERY': group['SURGERY'].sum(),
                            'Age 20-40': group["Age 20-40"].sum(), 'Age 40-60': group["Age 40-60"].sum(), 'Age 60-80':group["Age 60-80"].sum(), 
                            'Age 80-100':group["Age 80-100"].sum(), 'Age 100-120':group["Age 100-120"].sum(), "age_mean": [age_mean], "age_std": [age_std], 
                            "age_min": [min_age], "age_max": [max_age], "stay_min": [min_stay], "stay_max": [max_stay], "stay_mean": [stay_mean],
                            "stay_std": [stay_std], "freq": [round(freq,2)], 'Medical': group["Medical"].sum(), 'Mental':group["Mental"].sum(), 
                            'Others_Specialty':group["Others_Specialty"].sum(), 'Rehab': group["Rehab"].sum(), 'Gerontology': group["Gerontology"].sum(),
                            "died_by_cvd": died_by_cvd
                            })
    df = df.reset_index(drop=True)
    return df

In [7]:
tidy_dataset = []
for ids, group in tqdm(in_ad.groupby("Internalpatientid")):
    group = group.sort_values(by = ['Admission date'],ascending=True).reset_index(drop = True)
    df = final_set_before(ids, group)
    tidy_dataset.append(df)
    
tidy_dataset = pd.concat(tidy_dataset)

  0%|          | 0/22255 [00:00<?, ?it/s]

In [8]:
tidy_dataset = tidy_dataset.merge(death, how="left",on="Internalpatientid")

In [9]:
# we only want dead patients
tidy_dataset["Died"] = [1] * len(tidy_dataset)
tidy_dataset.loc[tidy_dataset["Age at death"].isna(), "Died"] = 0
tidy_dataset = tidy_dataset[tidy_dataset["Died"] == 1]

In [10]:
tidy_dataset = tidy_dataset.drop(["Died", "Death date"], axis=1)

In [11]:
tidy_dataset

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,AO,CVD,unique_admitting_specialty,unique_discharging_specialty,...,stay_mean,stay_std,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,died_by_cvd,Age at death
1,9,5,25.29,2,0,0,0,0,4,4,...,5.058000,5.481548,5.00,0,0,0,5,0,0,51.878258
2,12,6,24.37,1,0,5,1,1,2,3,...,4.061667,3.318231,1.20,6,0,0,0,0,0,74.122903
3,17,13,56.22,2,1,5,0,1,4,5,...,4.324615,3.370946,3.25,0,1,0,12,0,0,82.387875
4,22,3,13.20,2,1,0,0,0,2,1,...,4.400000,3.918125,0.25,0,0,0,3,0,0,61.533334
6,31,4,59.75,1,0,0,0,0,2,2,...,14.937500,18.290310,0.67,0,0,0,1,3,0,69.475314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22250,168995,3,4.68,3,0,0,0,0,3,2,...,1.560000,1.154253,0.23,1,0,0,1,1,0,75.662580
22251,169011,2,6.67,2,0,0,1,0,2,2,...,3.335000,3.599174,0.22,1,0,0,1,0,0,75.187690
22252,169037,3,66.82,3,0,0,0,0,2,2,...,22.273333,27.531904,1.50,2,0,0,1,0,0,87.743394
22253,169059,1,8.91,1,0,0,0,0,1,1,...,8.910000,0.000000,1.00,0,0,0,0,1,0,91.156817


In [12]:
if datatype == "train": tidy_dataset.to_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality.csv")
if datatype == "test": tidy_dataset.to_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality_test.csv")